In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gumbel_r

In [2]:
full=pd.read_csv('..\\..\\..\\Fermi-LAT Data\\hdu1.csv',index_col=0)
y_pred=pd.DataFrame()
y_proba=pd.DataFrame()

to_avg=pd.DataFrame()

for i in range(1,11):
    tempdf = pd.read_csv('Final Results\\y_results_imputed_wrappered_12_run'+str(i)+'.csv',index_col=0)
    to_avg['y_proba_1_it'+str(i)] = tempdf['y_proba_1']
    y_pred['it'+str(i)]=tempdf['y_pred']

data=pd.DataFrame(to_avg.mean(axis=1),columns=['y_proba_1'],index=tempdf.index)

'''
 In 'data' all unknown BCUs are assigned 0. 
        All BLLs are assigned 1 and all FSRQs are assigned 2.
        Radio Galaxies = 3 and Other = 4

        In the machine learning algorithm '0' corresponds to a BLL and '1' corresponds 
        to an FSRQ.
'''

gumbel_vals = gumbel_r.pdf(data['y_proba_1'],0.3421238340282159,0.19413067816085203)
data['Fit Values']=gumbel_vals

rdgrank=pd.merge(data,full[['Source_Name','RAJ2000','DEJ2000','ASSOC1','ASSOC2']],how='inner',
         left_index=True,right_index=True)
rdgrank=rdgrank.sort_values('Fit Values',ascending=False)
rdgrank['Ranking']=np.arange(0,len(rdgrank),1)

rdgrank_ltx=pd.merge(data,full[['Source_Name','RAJ2000','DEJ2000','ASSOC1','ASSOC_PROB_BAY','ASSOC_PROB_LR']],
                     how='inner',left_index=True,right_index=True)
rdgrank_ltx=rdgrank_ltx.drop('y_proba_1',axis=1)
#rdgrank_ltx=rdgrank_ltx.drop('Fit Values',axis=1)
rdgrank_ltx=rdgrank_ltx.sort_values('Fit Values',ascending=False)
rdgrank_ltx=rdgrank_ltx.head(5)
rdgrank_ltx=rdgrank_ltx.drop('Fit Values',axis=1)
rdgrank_ltx.to_latex('latex_tables\\RDG_Ranked_top_5',index=False,float_format="{:.2f}".format,)

In [3]:
from astroquery.ipac.ned import Ned
from tqdm import tqdm
from astropy.table import Table
from astropy.io import fits,ascii

N=20

for i in tqdm(range(0,50)):
    candidate = rdgrank['ASSOC1'].loc[rdgrank['Ranking']==i]
    fl=str(rdgrank['Source_Name'].loc[rdgrank['Ranking']==i])
    imageurl = Ned.get_image_list(candidate)
    image_toplot=Ned.get_images(candidate)
    fl= [idx for idx in fl.split(' ')]
    fl=fl[4]+fl[5]
    fl= [idx for idx in fl.split('\n')]
    fl=fl[0]
    hdu1 = fits.open(imageurl[0])
    pltscale=hdu1[0].header['PLTSCALE']
    xpixelsize=hdu1[0].header['XPIXELSZ']
    xpixelsizemm=xpixelsize*0.001
    pixelscale=pltscale*xpixelsizemm
    exposuretimeinminutes=hdu1[0].header['EXPOSURE']
    #print(exposuretime)
    print(image_toplot)
    for x in range(0,len(image_toplot)):
        image_data=image_toplot[0][x].data
        
        fig, ax = plt.subplots(figsize=(9,8),dpi=300)
        plt.title(f'Pixel Scale: {np.round(pixelscale,decimals=2)} arcsec/pixel',fontsize=N)
        plt.xticks(fontsize=N)
        plt.yticks(fontsize=N)
        plt.imshow(image_data, cmap='gray')
        cbar=plt.colorbar()
        cbar.ax.tick_params(labelsize=N)
        ax.tick_params('x',rotation=90)
        ax.set_xlabel('X (pixels)', fontsize=N)
        ax.set_ylabel('Y (pixels)', fontsize=N)
        cbar.set_label(f'No. Counts ({np.round(exposuretimeinminutes,decimals=2)} minute exposure)',
                        rotation=90,fontsize=N,labelpad=20)
        plt.savefig(f'Images\\{str(fl)}_image_'+str(x)+'.png',dpi=300)
        plt.subplots_adjust(bottom=0.2,left=0.1)
        #plt.show()
        plt.clf()
    #print(candidate)
hdu1[0].header

  0%|          | 0/50 [00:00<?, ?it/s]

4021    PKS 1452-367
Name: ASSOC1, dtype: object
[[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x0000024BEEE1D750>]]


  2%|▏         | 1/50 [00:00<00:38,  1.27it/s]

3046    PMN J1143-1017
Name: ASSOC1, dtype: object
[[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x0000024BEEF74B90>]]


  4%|▍         | 2/50 [00:01<00:35,  1.36it/s]

...

<Figure size 2700x2400 with 0 Axes>

<Figure size 2700x2400 with 0 Axes>

<Figure size 2700x2400 with 0 Axes>

<Figure size 2700x2400 with 0 Axes>

<Figure size 2700x2400 with 0 Axes>

In [4]:
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
from astropy.utils.data import get_pkg_data_filename
plt.style.use(astropy_mpl_style)


#image_file = image[0][0].data
#image_data=image_file
image_data=fits.info(image[0])
plt.figure()
plt.imshow(image_data, cmap='gray')
plt.colorbar()
plt.show()

NameError: name 'image' is not defined